<a href="https://colab.research.google.com/github/somesh-awasthi/NLP-PROJECT/blob/main/NLP_Project_V1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip3 install langchain langchain-openai
!pip3 install ctransformers sentence-transformers langchain-chroma
!pip3 install pandas nltk spacy PyPDF unstructured
!pip3 install poppler-utils pdf2image pdfminer.six
!pip3 install tesserocr pillow-heif
%pip install --upgrade --quiet  sentence-transformers langchain-chroma langchain langchain-openai > /dev/null

  Using cached tesserocr-2.7.0.tar.gz (71 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      pkg-config failed to find tesseract/leptonica libraries: Package 'tesseract' was not found
      
      Failed to extract tesseract version from executable: [WinError 2] The system cannot find the file specified
      Supporting tesseract v3.04.00
      Tesseract major version 3
      Traceback (most recent call last):
        File "<string>", line 239, in get_build_args
        File "<string>", line 120, in package_config
      Exception: Package 'tesseract' was not found
      
      
      During handling of the above exception, another exception occurred:
      
      Traceback (most recent call last):
        File "C:\Users\somes\venv-jupyterenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\somes\venv-jupyterenv\Lib\site-packages\pip\_ve

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


In [ ]:
# connecting to database
# from google.colab import drive
# drive.mount("/content/drive")
# path="/content/drive/MyDrive/Colab Notebooks/data"
path = "../data"

In [ ]:
from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
def partition_doc():
    raw_pdf_elements = partition_pdf(
        filename= path + "/symp.pdf",
        extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
    )

    # Categorize by type
    tables = []
    texts = []

    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))

    print(len(tables), " ", len(texts))
    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
    Give a concise summary of the table or text. Table or text chunk: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Summary chain
    model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return table_summaries, texts

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
# Load documents from PDF
loader = DirectoryLoader(path, glob="symp.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords

nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Tokenization and POS tagging using SpaCy
    doc = nlp(text)

    # Filtering out tokens based on POS tags and dependency parsing
    filtered_tokens = [token.text.lower() for token in doc if token.pos_ not in ["SPACE", "X"] and token.dep_ not in ["det", "punct"]]

    # Stopword removal
    filtered_tokens = [token for token in filtered_tokens if token not in stopwords.words('english')]

    # Lemmatization
    lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(filtered_tokens))]

    return " ".join(lemmatized_tokens)

In [ ]:
# Preprocess each document
for doc in documents:
    doc.page_content = preprocess_text(doc.page_content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split the preprocessed documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

In [ ]:
# Save preprocessed chunks to Chroma

import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LANGCHAIN API key:')

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# db = Chroma.from_documents(chunks, OpenAIEmbeddings(), persist_directory="./drive/MyDrive/Colab Notebooks/chroma_db")
db = Chroma.from_documents(chunks, OpenAIEmbeddings(), persist_directory="../chroma_db-v5")
# db = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="../chroma_db-v1")

In [ ]:
from langchain.prompts import ChatPromptTemplate
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

---

Answer the question based on the above context: {question}
Only return the helpful answer below and nothing else.
"""

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_community.document_transformers import (
    LongContextReorder,
)
# Start conversation loop
context_text = ""
while True:
    query_text = input("Enter your query (type 'quit' to exit): ")

    if query_text.lower() == 'quit':
        break

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=7)

    # Reorder documents
    reordering = LongContextReorder()
    reorder_docs = reordering.transform_documents(results)

    if len(reorder_docs) == 0 or reorder_docs[0][1] < 0.7:
        print(f"Unable to find matching results.")
        continue

    new_context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in reorder_docs])
    context_text += "\n\n---\n\n" + new_context_text
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = ChatOpenAI()
    response_text = model.invoke(prompt)

    # Load the model
    sources = [doc.metadata.get("source", None) for doc, _score in reorder_docs]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)